# Prelude

In [231]:
import pprint
from itertools import combinations, chain
import math

In [232]:
from IPython import get_ipython

def custom_float_formatter(float_value, p, cycle):
    p.text(f"{float_value:g}")

ip = get_ipython()
formatter = ip.display_formatter.formatters['text/plain']
formatter.for_type(float, custom_float_formatter)

<function __main__.custom_float_formatter(float_value, p, cycle)>

# First Formula

$$
V^{n + 1}_{A, \varrho} = V^n_{A, \varrho} + \alpha_A \, \beta_{\varrho} \, \left( \gamma_{\varrho} - \sum_{q = 0}^n { V^n_q } \right)
$$

In [27]:
alpha = dict(
    A = 0.1,
    B = 0.2,
    C = 0.3,
)

beta = 1
gamma = 1

In [72]:
use = alpha
V = {p: [0.] for p in use.keys()}

steps = 10
for i in range(steps):
    sigma = sum(x[-1] for x in V.values())
    for k, v in V.items():
        v.append(v[-1] + alpha[k] * beta * (gamma - sigma))

h = V.copy()
for q in range(2, len(V) + 1):
    for k in combinations(h.items(), q):
        V[''.join(x[0] for x in k)] = [sum(y) for y in zip(*(x[1] for x in k))]

In [239]:
class Group:
    def __init__(self, alphas, betan, betap, gamman, gammap, use_compounds = False):
        self.alphas = alphas
        self.betan = betan
        self.betap = betap
        self.gamman = gamman
        self.gammap = gammap
        self.use_compounds = use_compounds

        self.cs = [x for x in alphas.keys() if len(x) == 1]
        if use_compounds:
            simples = {k: v for k, v in alphas.items() if len(k) == 1}
            compounds = {
                ''.join(p[0] for p in x): math.prod(p[1] for p in x)
                for x in sum([list(combinations(simples.items(), t)) for t in range(1, len(simples) + 1)], [])
            }
            self.alphas = compounds | alphas
            self.cs = self.alphas.keys()

        self.assoc = {c: 0. for c in self.cs}

    @staticmethod
    def combine(V):
        h = dict()

        simples = {k: v for k, v in V.items() if len(k) == 1}
        compounds = {k: v for k, v in V.items() if len(k) > 1}
        for size in range(1, len(V) + 1):
            for comb in combinations(simples.items(), size):
                names = [x[0] for x in comb]
                assocs = [x[1] for x in comb]
                h[''.join(names)] = [sum(x) for x in zip(*assocs)]

        for k, v in compounds.items():
            h[k] = [x + y for x, y in zip(h[k], v)]
            h[f'c({k})'] = v

        return h

    def runPhase(self, steps, conds, plus = True, ret_V = False):
        beta = self.betan
        gamma = self.gamman
        if plus:
            beta = self.betap
            gamma = self.gammap

        compounds = list(conds)
        if self.use_compounds:
            compounds.append(conds)

        V = {x: [self.assoc[x]] for x in compounds}
        for step in range(steps):
            sigma = sum(self.assoc[x] for x in compounds)
            for cs in compounds:
                self.assoc[cs] += self.alphas[cs] * beta * (gamma - sigma)
                V[cs].append(self.assoc[cs])

        if ret_V:
            return self.combine(V)

        return self

In [207]:
g = Group(
    alphas = {'A': 0.2, 'B': 0.2, 'C': 0.2},
    betan = 0.5,
    gamman = 0,
    betap = 1,
    gammap = 1,
)

v0 = g.runPhase(5, ['A', 'B'], ret_V = True)
v1 = g.runPhase(5, ['A', 'B'], plus = False, ret_V = True)

[v0, v1]

[{'A': [0, 0.2, 0.32, 0.392, 0.4352, 0.46112],
  'B': [0, 0.2, 0.32, 0.392, 0.4352, 0.46112],
  'AB': [0, 0.4, 0.64, 0.784, 0.8704, 0.92224]},
 {'A': [0.46112, 0.368896, 0.295117, 0.236093, 0.188875, 0.1511],
  'B': [0.46112, 0.368896, 0.295117, 0.236093, 0.188875, 0.1511],
  'AB': [0.92224, 0.737792, 0.590234, 0.472187, 0.37775, 0.3022]}]

In [240]:
g = Group(
    alphas = {'A': 0.2, 'B': 0.2, 'C': 0.2},
    betan = 0.5,
    gamman = 0,
    betap = 1,
    gammap = 1,
    use_compounds = False,
)

v0 = g.runPhase(5, 'ABC', ret_V = True)
v1 = g.runPhase(5, 'AB', plus = False, ret_V = True)

[v0, v1]

[{'A': [0, 0.2, 0.28, 0.312, 0.3248, 0.32992],
  'B': [0, 0.2, 0.28, 0.312, 0.3248, 0.32992],
  'C': [0, 0.2, 0.28, 0.312, 0.3248, 0.32992],
  'AB': [0, 0.4, 0.56, 0.624, 0.6496, 0.65984],
  'AC': [0, 0.4, 0.56, 0.624, 0.6496, 0.65984],
  'BC': [0, 0.4, 0.56, 0.624, 0.6496, 0.65984],
  'ABC': [0, 0.6, 0.84, 0.936, 0.9744, 0.98976]},
 {'A': [0.32992, 0.263936, 0.211149, 0.168919, 0.135135, 0.108108],
  'B': [0.32992, 0.263936, 0.211149, 0.168919, 0.135135, 0.108108],
  'AB': [0.65984, 0.527872, 0.422298, 0.337838, 0.27027, 0.216216]}]

In [ ]:
g = Group(
    alphas = {'A': 0.2, 'B': 0.2, 'AB': 0.2 ** 2, 'C': },
    betan = 0.5,
    gamman = 0,
    betap = 1,
    gammap = 1,
    use_compounds = True,
)

v0 = g.runPhase(5, 'AB', ret_V = True)
v1 = g.runPhase(5, 'AB', plus = False, ret_V = True)

[v0, v1]